In [19]:
import pandas as pd
import numpy as np

# Dropping Columns
First we will load the combined csv file as a dataframe, before dropping columns which are irrelevant to our questions.

In [20]:
combined_df = pd.read_csv("../data/processed/combined_ATP_results.csv")
combined_df.head()

,Unnamed: 0,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,0,1998-339,Adelaide,Hard,32,A,19980105,1,102035,1.0,...,29.0,18.0,11.0,9.0,2.0,5.0,4.0,2949.0,74.0,649.0
1,1,1998-339,Adelaide,Hard,32,A,19980105,2,101727,NaN,...,32.0,19.0,11.0,8.0,9.0,13.0,79.0,617.0,87.0,537.0
2,2,1998-339,Adelaide,Hard,32,A,19980105,3,102765,NaN,...,38.0,18.0,7.0,9.0,5.0,12.0,93.0,521.0,71.0,665.0
3,3,1998-339,Adelaide,Hard,32,A,19980105,4,102563,7.0,...,37.0,21.0,10.0,11.0,1.0,6.0,39.0,959.0,76.0,633.0
4,4,1998-339,Adelaide,Hard,32,A,19980105,5,102796,4.0,...,57.0,33.0,20.0,13.0,12.0,17.0,22.0,1450.0,65.0,708.0


I have highlighted the following columns as not being required:
* 'Unnamed: 0' - placeholder
* 'tourney_id' - tournament id code; the name and level will be much more useful, and conveys the same information
* 'match_num' - inconsitent data, which is sometimes arbituary
* 'winner_id', 'loser_id' - same information conveyed by the name
* 'winner_seed', 'loser_seed' - too much missing data
* 'winner_entry', 'loser_entry' - as above
* 'winner_hand', 'winner_ht', 'winner_ioc', 'loser_hand', 'loser_ht', 'loser_ioc' - irrelevant data to the question; these are abrituary metrics for performance (unlike age)
* 'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon', 'w_SvGms', 'w_bpSaved', 'w_bpFaced' (and the same for 'l_') - match specific data not needed within a broader quesiton; matches do not need to be analysed as in depth as this; 'score' and 'best_of' are being kept as they are not mising values, and can be used to measure 'closeness' of a match (useful for later network analysis)

In [21]:
columns_to_drop = [
    'Unnamed: 0', 'tourney_id', 'match_num', 'winner_id', 'loser_id',
    'winner_seed', 'loser_seed', 'winner_entry', 'loser_entry', 
    'winner_hand', 'winner_ht', 'winner_ioc', 'loser_hand', 'loser_ht', 'loser_ioc', 'minutes',
    'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon', 'w_SvGms', 'w_bpSaved', 'w_bpFaced',
    'l_ace', 'l_df', 'l_svpt', 'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced'
]
cleaned_df = combined_df.drop(columns=columns_to_drop)
cleaned_df.head()

,tourney_name,surface,draw_size,tourney_level,tourney_date,winner_name,winner_age,loser_name,loser_age,score,best_of,round,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,Adelaide,Hard,32,A,19980105,Jonas Bjorkman,25.7,Grant Stafford,26.6,6-4 6-2,3,R32,4.0,2949.0,74.0,649.0
1,Adelaide,Hard,32,A,19980105,Jason Stoltenberg,27.7,Juan Antonio Marin,22.8,6-4 6-1,3,R32,79.0,617.0,87.0,537.0
2,Adelaide,Hard,32,A,19980105,Nicolas Escude,21.7,Alex Radulescu,23.0,6-0 7-5,3,R32,93.0,521.0,71.0,665.0
3,Adelaide,Hard,32,A,19980105,Thomas Johansson,22.7,Byron Black,28.2,7-5 6-3,3,R32,39.0,959.0,76.0,633.0
4,Adelaide,Hard,32,A,19980105,Magnus Norman,21.6,Christian Ruud,25.3,6-3 1-6 6-4,3,R32,22.0,1450.0,65.0,708.0


# Dealing with Missing Data
Some of the remaining columns still have missing data which needs dealt with.

In [22]:
cleaned_df.isnull().sum()

tourney_name             0
surface                 53
draw_size                0
tourney_level            0
tourney_date             0
winner_name              0
winner_age               5
loser_name               0
loser_age                3
score                    0
best_of                  0
round                    0
winner_rank            703
winner_rank_points     703
loser_rank            1683
loser_rank_points     1686
dtype: int64

With the small amount of missing data for surface (which will likely be across 1 or 2 tournaments and hence the same result missing for many entries), and player ages, I will start by locating these and manually adding them.

In [33]:
missing_surfaces = cleaned_df[cleaned_df['surface'].isna()]
print(missing_surfaces)

Empty DataFrame
Columns: [tourney_name, surface, draw_size, tourney_level, tourney_date, winner_name, winner_age, loser_name, loser_age, score, best_of, round, winner_rank, winner_rank_points, loser_rank, loser_rank_points]
Index: []


All missing surface data is form the same tournament, one in which none of our players of interest played. I therefore see it fit to remove all rows with missing surface data.

In [31]:
cleaned_df1 = cleaned_df.dropna(subset=['surface'])
cleaned_df1.isnull().sum()

tourney_name             0
surface                  0
draw_size                0
tourney_level            0
tourney_date             0
winner_name              0
winner_age               5
loser_name               0
loser_age                2
score                    0
best_of                  0
round                    0
winner_rank            695
winner_rank_points     695
loser_rank            1671
loser_rank_points     1674
dtype: int64

In [32]:
cleaned_df = cleaned_df1

We will now locate the missing ages and add them accordingly.

In [34]:
missing_ages = cleaned_df[cleaned_df['winner_age'].isna() | cleaned_df['loser_age'].isna()]
print(missing_ages)

                       tourney_name surface  draw_size tourney_level  \
10040   Davis Cup G1 PO: LIB vs CHN    Hard          4             D   
16793   Davis Cup G2 PO: GHA vs ARM    Hard          4             D   
68563   Davis Cup G2 R1: NZL vs INA    Hard          4             D   
75668  Davis Cup WG1 R1: BIH vs MEX    Clay          4             D   
78695  Davis Cup WG2 R1: ESA vs IRL    Clay          4             D   
81800  Davis Cup WG2 PO: CYP vs MAR    Hard          4             D   
81801  Davis Cup WG2 PO: ESA vs POC    Hard          4             D   

       tourney_date                 winner_name  winner_age        loser_name  \
10040      20001006                      Y Wang         NaN       Ali Hamadeh   
16793      20020712                     F Egyir         NaN  Tsolak Gevorgyan   
68563      20190914                 Ari Fahresi         NaN         Ajeet Rai   
75668      20220916  Luis Carlos Alvarez Valdes         NaN      Andrej Nedic   
78695      2023091

Once again as no players of interest are affected by the missing age data, we shall drop the rows.

In [35]:
cleaned_df.dropna(subset=['winner_age', 'loser_age'], inplace=True)
cleaned_df.isnull().sum()

tourney_name             0
surface                  0
draw_size                0
tourney_level            0
tourney_date             0
winner_name              0
winner_age               0
loser_name               0
loser_age                0
score                    0
best_of                  0
round                    0
winner_rank            690
winner_rank_points     690
loser_rank            1666
loser_rank_points     1669
dtype: int64

Finally we have to deal with missing data in the rankings. There is too much missing to warrant looking through manually, so we must look at what the data means, and whether or not we can drop missing data, or if we can reliably extrapolate.

In [43]:
players_of_interest = ['Roger Federer', 'Rafael Nadal', 'Novak Djokovic', 'Andy Murray']
cleaned_df[cleaned_df['winner_name'].isin(players_of_interest) | cleaned_df['loser_name'].isin(players_of_interest)].isna().sum()

tourney_name           0
surface                0
draw_size              0
tourney_level          0
tourney_date           0
winner_name            0
winner_age             0
loser_name             0
loser_age              0
score                  0
best_of                0
round                  0
winner_rank            0
winner_rank_points     0
loser_rank            10
loser_rank_points     10
dtype: int64

As we can see dropping na values is not an option as there are missing values within our players of interest matches. Rankings would be useful as a measure of consistency, but this is impractical with missing data. Instead I will introduce an ELO system, allowing us to measure not only consistency, but also peak performance and length of primes. We can also keep track of surface specific ELO, as the surface can drastically change the outcome of a game. In light of this, we can remove all variables related to rank from our data.

In [44]:
cleaned_df = cleaned_df.drop(columns=['winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points'])
cleaned_df.head()

,tourney_name,surface,draw_size,tourney_level,tourney_date,winner_name,winner_age,loser_name,loser_age,score,best_of,round
0,Adelaide,Hard,32,A,19980105,Jonas Bjorkman,25.7,Grant Stafford,26.6,6-4 6-2,3,R32
1,Adelaide,Hard,32,A,19980105,Jason Stoltenberg,27.7,Juan Antonio Marin,22.8,6-4 6-1,3,R32
2,Adelaide,Hard,32,A,19980105,Nicolas Escude,21.7,Alex Radulescu,23.0,6-0 7-5,3,R32
3,Adelaide,Hard,32,A,19980105,Thomas Johansson,22.7,Byron Black,28.2,7-5 6-3,3,R32
4,Adelaide,Hard,32,A,19980105,Magnus Norman,21.6,Christian Ruud,25.3,6-3 1-6 6-4,3,R32


In [45]:
cleaned_df.isnull().sum()

tourney_name     0
surface          0
draw_size        0
tourney_level    0
tourney_date     0
winner_name      0
winner_age       0
loser_name       0
loser_age        0
score            0
best_of          0
round            0
dtype: int64

In [46]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 81771 entries, 0 to 81830
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   tourney_name   81771 non-null  object 
 1   surface        81771 non-null  object 
 2   draw_size      81771 non-null  int64  
 3   tourney_level  81771 non-null  object 
 4   tourney_date   81771 non-null  int64  
 5   winner_name    81771 non-null  object 
 6   winner_age     81771 non-null  float64
 7   loser_name     81771 non-null  object 
 8   loser_age      81771 non-null  float64
 9   score          81771 non-null  object 
 10  best_of        81771 non-null  int64  
 11  round          81771 non-null  object 
dtypes: float64(2), int64(3), object(7)
memory usage: 8.1+ MB


We now have a cleaned dataframe with 81771 matches, with succinct data and no missing values. Our final step is to save this cleaned dataframe as a csv file.

In [47]:
cleaned_df.to_csv("../data/processed/cleaned_ATP_results.csv", index=False)